# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на тарифы оператора "Мегалайн" - "Ультра" и "Смарт". Нужно построить модель для задачи классификации, которая выберет подходящий для абонента тариф. Предобработка данных не понадобится — она уже была выполнена в более раннем проекте.

Необходимо построить модель с максимально большим значением *accuracy*. Цель - довести долю правильных ответов по крайней мере до 0.75.

## Откройте и изучите файл

In [1]:
# импортируем все необходимое
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
# читаем таблицу и сохраняем ее в переменную df
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
# выведем первые 5 строк таблицы
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
# посмотрим информацию по типу данных в столбцах и кол-ву пропусков, если они есть
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


В столбцах `messages` и `calls` можем заменить тип данных на int, т.к. в данных столбцах могут быть только целые значения.

In [5]:
# заменим тип данных в столбцах с кол-вом сообщений и звонков на int
df['messages'] = df['messages'].astype("int")
df['calls'] = df['calls'].astype("int")

И, если я все правильно понимаю, в рамках данного исследования нам не нужно округлять значения израсходованных пакетов услуг (как это на практике делает оператор), т.к. от нас не требуется вычислять, сколько каждый абонент тратит на услуги связи и, соответственно, приносит прибыли компании. Поэтому дополнительная предобработка значений не требуется.

In [6]:
# проверим, есть ли явные дубликаты
df.duplicated().sum()

0

**Выводы:**

- Данные полные, пропусков нет;
- Выборка небольшая, чуть больше трех тысяч пользователей;
- Корректные наименования столбцов (все в нижнем регистре, нет лишних пробелов и т.д.);
- Нет дубликатов.

## Разбейте данные на выборки

Для начала разобьем датафрейм на признаки и целевой признак. Целевым признаком у нас будет столбец с информацией о тарифе абонента: "сидит" на тарифе "Ультра" или нет. Соответственно, раз у нас выделен целевой признак, обучение модели будет проходить "с учителем".

In [7]:
# разделим датафрейм на features - все признаки, кроме целевого и target - целевой признак
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

Далее разделим данные на 3 выборки - обучающую, тестовую и валидацинную в пропорции 3:1:1, т.е. 60%, 20% и 20%.

In [8]:
# сначала разделим данные на обучающую и тестовую выборки. В random_state запишем 12345
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.4, random_state=12345)

In [9]:
# теперь разделим тестовую выборку на тестовую и валидационную:
features_test, features_valid, target_test, target_valid = train_test_split(features_test, target_test, test_size=0.5, random_state=12345)

## Исследуйте модели

В рамках теории нами были изучены три модели - "Случайный лес", "Дерево решений" и "Линейная регрессия". Как я понимаю, две из них (Лес и Дерево) используются как раз таки для решения задач классификации, "Линейная регрессия" - для задач регрессии (однако нашла пару статей, где говорится, что в теории можно использовать и для бинарной классификации, как у нас в исследовании, но показали далеки от адекватности). Поэтому будем рассматривать две модели - "Случайный лес" и "Дерево Решений".

С помощью цикла найдем для каждой модели оптимальное соотношение заданных значений гиперпараметров, чтобы оценить максимальное значение точности для каждой модели и выбрать наилучшую.

### Random Forest

In [10]:
best_model_RF = None
best_accuracy_RF = 0
best_est_RF = 0
best_depth_RF = 0
for est in range(2,51):
    for depth in range(2,25):
        model_RF = RandomForestClassifier(random_state = 12345, n_estimators = est, max_depth = depth)
        model_RF.fit(features_train, target_train)
        prediction_valid_RF = model_RF.predict(features_valid)
        accuracy_RF = accuracy_score(prediction_valid_RF, target_valid)
        if best_accuracy_RF < accuracy_RF:
            best_model_RF = model_RF
            best_depth_RF = depth
            best_est_RF = est
            best_accuracy_RF = accuracy_RF

In [11]:
print(f"Наибольшая точность модели Random Forest: {best_accuracy_RF}")

Наибольшая точность модели Random Forest: 0.8149300155520995


### Decision Tree

In [12]:
best_model_DT = None
best_accuracy_DT = 0
best_depth_DT = 0
for depth in range(2,50):
    model_DT = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
    model_DT.fit(features_train, target_train)
    prediction_valid_DT = model_DT.predict(features_valid)
    accuracy_DT = accuracy_score(prediction_valid_DT, target_valid)
    if best_accuracy_DT < accuracy_DT:
        best_model_DT = model_DT
        best_accuracy_DT = accuracy_DT
        best_depth_DT = depth

In [13]:
print(f"Наибольшая точность модели Decision Tree: {best_accuracy_DT}")

Наибольшая точность модели Decision Tree: 0.7993779160186625


**Вывод:**

Основываясь на показателе точности, можно прийти к выводу, что наиболее качественные результаты прогнозирует Random Forest

## Проверьте модель на тестовой выборке

In [14]:
# получим предсказания обеих моделей на тестовой выборке
prediction_test_RF = best_model_RF.predict(features_test)
prediction_test_DT = best_model_DT.predict(features_test)

In [15]:
# оценим точность предсказаний каждой модели
accuracy_RF_test = accuracy_score(prediction_test_RF, target_test)
accuracy_DT_test = accuracy_score(prediction_test_DT, target_test)

In [16]:
# выведем на экран результаты
print(f"Random Forest: {accuracy_RF_test}")
print(f"Decision Tree: {accuracy_DT_test}")

Random Forest: 0.7900466562986003
Decision Tree: 0.7822706065318819


**Вывод:**

По точности предсказаний на валидационной и на тестовой выборках, можем утверждать, что модель Random Forest делает наиболее адекватные предсказания.    

## (бонус) Проверьте модели на адекватность

Для проверки модели на адекватность сравним точность предсказаний модели Random Forest с точностью предсказаний случайной модели на основе инструмента той же библиотеки `sklearn` - `DummyClassifier`.

In [17]:
# обучим случайную модель на обучающей выборке
dummy_csf = DummyClassifier(strategy='most_frequent').fit(features_train, target_train)
# получим предсказания от случайной модели на валидационной и тестовой выборках
dummy_prediction_valid = dummy_csf.predict(features_valid)
dummy_prediction_test = dummy_csf.predict(features_test)
# оценим точность предсказаний на обеих выборках
dummy_accuracy_valid = accuracy_score(dummy_prediction_valid, target_valid)
dummy_accuracy_test = accuracy_score(dummy_prediction_test, target_test)
# выведем результаты
print(f"Точность предсказаний случайной модели на валидационной выборке: {dummy_accuracy_valid}")
print(f"Точность предсказаний случайной модели на тестовой выборке: {dummy_accuracy_test}")

Точность предсказаний случайной модели на валидационной выборке: 0.6842923794712286
Точность предсказаний случайной модели на тестовой выборке: 0.7060653188180405


Т.к. точность предсканий модели Random Forest была выше как на валидационной, так и на тестовой выборке, делаем вывод о том, что модель делает адекватные предсказания.